In [ ]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from numpy import mean, absolute
import openpyxl
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot
from tensorflow.keras.models import load_model
import warnings

# Ignore the warning message
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.metrics import (
    log_loss,
    confusion_matrix,
    roc_auc_score,
    recall_score,
    precision_score,
    average_precision_score,
    f1_score,
    classification_report,
    accuracy_score,
    plot_roc_curve,
    plot_precision_recall_curve,
    plot_confusion_matrix,
)

In [ ]:
# Import feature list
infile = open(
    "../01_solution_probability_model/data_solution_probability_model.pkl", "rb"
)
import_file = pickle.load(infile)
infile.close()
df = import_file
feature_cols = list(df.columns)

In [ ]:
def get_metrics(clf, X_train, y_train, X_test, y_test, cv):
    """
    function to predict y and to calculate all scores neccessary for fairness evaluation
    """
    scores_a = cross_val_score(
        clf, X_train, y_train, scoring="accuracy", cv=cv, n_jobs=-1
    )
    accuracy = mean(absolute(scores_a))
    scores_p = cross_val_score(
        clf, X_train, y_train, scoring="precision", cv=cv, n_jobs=-1
    )
    precision = mean(absolute(scores_p))
    scores_r = cross_val_score(
        clf, X_train, y_train, scoring="recall", cv=cv, n_jobs=-1
    )
    recall = mean(absolute(scores_r))
    scores_auc = cross_val_score(
        clf, X_train, y_train, scoring="roc_auc", cv=cv, n_jobs=-1
    )
    roc_auc_cv = mean(absolute(scores_auc))

    pred = clf.predict(X_test)
    a = accuracy_score(y_test, pred)
    p = precision_score(y_test, pred)
    r = recall_score(y_test, pred)
    roc_auc = roc_auc_score(y_test, pred)
    tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
    fpr = fp / (fp + tn)

    return a, p, r, roc_auc, fpr

In [ ]:
## list subgroups
fair_metrics = pd.DataFrame(
    columns=[
        "model",
        "group",
        "subgroup",
        "Accuracy",
        "Precision",
        "Recall",
        "AUC",
        "FPR",
    ]
)
matrice = [
    "double_df_abi",
    "double_df_keinAbi",
    "double_df_boys",
    "double_df_girls",
    "double_df_deutsch",
    "double_df_migration",
    "double_df_buch0",
    "double_df_buch1",
]
group = [
    "abiEltern",
    "abiEltern",
    "gender",
    "gender",
    "erstsprache",
    "erstsprache",
    "buecher",
    "buecher",
]
subgroup = ["abi", "keinAbi", "boys", "girls", "deutsch", "migration", "buch0", "buch1"]

Decision Tree Classifier

In [ ]:
# load model
DTE_model = pickle.load(
    open(
        "../01_solution_probability_model/01_decision_tree/Decisiontreemodel_3months.pkl",
        "rb",
    )
)

# predict for each subgroup
for group, subgroup, matrice in zip(group, subgroup, matrice):
    print(matrice)
    path = "matrices_subgroups/" + matrice + ".pkl"
    infile = open(path, "rb")
    df = pickle.load(infile)
    infile.close()

    dataset = df[df.columns[df.columns.isin(feature_cols)]]
    y = dataset["Erfolg"]
    X = dataset.drop(columns=["Erfolg"])
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=1
    )
    k = 5
    cv = KFold(n_splits=k, random_state=None)
    a, p, r, roc_auc, fpr = get_metrics(DTE_model, X_train, y_train, X_test, y_test, cv)
    fair_metrics = fair_metrics.append(
        {
            "model": "DTE",
            "group": group,
            "subgroup": subgroup,
            "Accuracy": a,
            "Precision": p,
            "Recall": r,
            "AUC": roc_auc,
            "FPR": fpr,
        },
        ignore_index=True,
    )

fair_metrics.to_excel("metrics/dte_metrics.xlsx")
fair_metrics.to_pickle("metrics/dte_metrics.pkl")

Logistic Regression

In [ ]:
# load model
logreg_model = pickle.load(
    open(
        "../01_solution_probability_model/02_logistic_regression/Logregmodel_3months.pkl",
        "rb",
    )
)

# predict for each subgroup
for group, subgroup, matrice in zip(group, subgroup, matrice):
    path = matrice + ".pkl"
    infile = open(path, "rb")
    df = pickle.load(infile)
    infile.close()

    dataset = df[df.columns[df.columns.isin(feature_cols)]]
    y = dataset["Erfolg"]
    X = dataset.drop(columns=["Erfolg"])
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=1
    )
    k = 5
    cv = KFold(n_splits=k, random_state=None)
    a, p, r, roc_auc, fpr = get_metrics(
        logreg_model, X_train, y_train, X_test, y_test, cv
    )
    fair_metrics = fair_metrics.append(
        {
            "model": "LogReg",
            "group": group,
            "subgroup": subgroup,
            "Accuracy": a,
            "Precision": p,
            "Recall": r,
            "AUC": roc_auc,
            "FPR": fpr,
        },
        ignore_index=True,
    )

fair_metrics.to_excel("metrics/log_metrics.xlsx")
fair_metrics.to_pickle("metrics/log_metrics.pkl")

SVM

In [ ]:
# load model
svm_model = pickle.load(
    open("../01_solution_probability_model/03_svm/SVMmodel_3months.pkl", "rb")
)

# predict for each subgroup
for group, subgroup, matrice in zip(group, subgroup, matrice):
    path = matrice + ".pkl"
    infile = open(path, "rb")
    df = pickle.load(infile)
    infile.close()

    dataset = df[df.columns[df.columns.isin(feature_cols)]]
    y = dataset["Erfolg"]
    X = dataset.drop(columns=["Erfolg"])
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=1
    )
    k = 5
    cv = KFold(n_splits=k, random_state=None)
    a, p, r, roc_auc, fpr = get_metrics(svm_model, X_train, y_train, X_test, y_test, cv)
    fair_metrics = fair_metrics.append(
        {
            "model": "SVM",
            "group": group,
            "subgroup": subgroup,
            "Accuracy": a,
            "Precision": p,
            "Recall": r,
            "AUC": roc_auc,
            "FPR": fpr,
        },
        ignore_index=True,
    )

fair_metrics.to_excel("metrics/svm_metrics.xlsx")
fair_metrics.to_pickle("metrics/svm_metrics.pkl")

Neural Network

In [ ]:
def get_dn_metrics(model, X_test, y_test):
    """
    function to predict y in nn model
    return all metrics neccessary for fairness evaluation
    """
    X_test = np.asarray(X_test).astype("float32")
    yhat_probs = model.predict(X_test, verbose=0)
    yhat_classes = (model.predict(X_test) > 0.5).astype("int32")
    # reduce to 1d array
    yhat_probs = yhat_probs[:, 0]
    yhat_classes = yhat_classes[:, 0]
    a = accuracy_score(y_test, yhat_classes)
    p = precision_score(y_test, yhat_classes)
    r = recall_score(y_test, yhat_classes)
    roc_auc = roc_auc_score(y_test, yhat_probs)
    tn, fp, fn, tp = confusion_matrix(y_test, yhat_classes).ravel()
    fpr = fp / (fp + tn)

    return a, p, r, roc_auc, fpr


# load model
nn_model = load_model("../01_solution_probability_model/04_nn/nn/")

# predict for each subgroup
for group, subgroup, matrice in zip(group, subgroup, matrice):
    path = matrice + ".pkl"
    infile = open(path, "rb")
    df = pickle.load(infile)
    infile.close()

    dataset = df[df.columns[df.columns.isin(feature_cols)]]
    y = dataset["Erfolg"]
    X = dataset.drop(columns=["Erfolg"])
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=1
    )
    a, p, r, roc_auc, fpr = get_dn_metrics(nn_model, X_test, y_test)
    fair_metrics = fair_metrics.append(
        {
            "model": "NN",
            "group": group,
            "subgroup": subgroup,
            "Accuracy": a,
            "Precision": p,
            "Recall": r,
            "AUC": roc_auc,
            "FPR": fpr,
        },
        ignore_index=True,
    )

fair_metrics.to_excel("metrics/nn_metrics.xlsx")
fair_metrics.to_pickle("metrics/nn_metrics.pkl")